In [2]:
import pandas as pd
import numpy
import os
from sklearn.ensemble import GradientBoostingRegressor
# from sklearn.inspection import permutation_importance
from sklearn import metrics
from sklearn.model_selection import cross_val_score, KFold, GridSearchCV
from sklearn.metrics import make_scorer, r2_score,  mean_squared_log_error, mean_absolute_error, mean_squared_error,mean_squared_log_error
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

In [3]:
uk_data =pd.read_excel('UK.xlsx')

In [68]:
germ_data =pd.read_excel('germ.xlsx')

In [69]:
germ_data.columns

Index(['Manufacturer Identification Number', 'Model Identification Numer',
       'No. Of Driving Axles', 'max. Length', 'max. Width', 'max. Height',
       'max. Gross Vehicle Weight', 'Manufacturer', 'Commercial Name', 'Type',
       'Wheelbase', 'Power/Weight Ratio', 'Seats', 'Fuel Code',
       'max. Top Speed', 'Fuel Type'],
      dtype='object')

In [71]:
germ_data.shape

(83615, 16)

In [72]:
uk_data.shape

(17856, 28)

In [70]:
uk_data.columns

Index(['Vehicle Type', 'Manufacturer', 'Model', 'Series', 'ABI Code',
       'Wheel Base Length', 'Roof Height Type', 'Number Of Seats',
       'Total Height', 'Total Length', 'Total Width', 'Kerb Weight',
       'Gross Weight', 'Max Payload', 'Torque', 'Max Speed', 'Acceleration',
       'Power Output', 'Fuel Type', 'Secondary Fuel Type', 'ABI Description',
       'Vehicle Import Type', 'AEB Type', 'List Price',
       'Enhanced Vehicle Description', 'Battery Lease', 'CO2 Emissions G/KM',
       'Electric Vehicle Type'],
      dtype='object')

In [7]:
n_missing = germ_data.isna().sum()
print(f"Number of missing values present in each feature are: \n{n_missing}")

Number of missing values present in each feature are: 
Manufacturer Identification Number        0
Model Identification Numer                0
No. Of Driving Axles                  18697
max. Length                           10791
max. Width                            10841
max. Height                           10989
max. Gross Vehicle Weight                 0
Manufacturer                          53155
Commercial Name                        1312
Type                                      6
Wheelbase                             15074
Power/Weight Ratio                    11188
Seats                                 13392
Fuel Code                             11865
max. Top Speed                        12410
Fuel Type                             11875
dtype: int64


In [73]:
germ_data['Type'].unique()

array(['Typ', 'M 100', 'MR 100;LUXUS MR100', ..., 'L20H', 'L25H',
       'EWR170'], dtype=object)

In [74]:
uk_data.rename(columns = {'Primary Fuel Type':'Fuel Type'},inplace = True)

In [75]:
germ_data=germ_data.drop(index=0,axis=0)

In [76]:
germ_data.shape

(83614, 16)

In [77]:
germ_data = germ_data.dropna(subset=['Fuel Type'])

In [78]:
germ_data.shape

(71739, 16)

In [79]:
germ_data['Fuel Type'].unique()

array(['Petrol', '\xa0Diesel', 'Hybrid Petrol/Electric', 'Electric',
       'Hybrid LPG/PHEV', 'Petrol/Compressed Natural Gas',
       'Multi-substance', 'Natural Gas', 'Petrol/Liquid gas',
       'Hybrid Petrol/PHEV', 'Hybrid Hydrogen/PHEV',
       'Hybrid Diesel/Electric', 'Hydrogen/FC', 'LPG',
       'Hybrid LPG/Electric', 'Hybrid Hydrogen/FC/PHEV'], dtype=object)

In [82]:
germ_data['Fuel Type'] = germ_data['Fuel Type'].str.upper()

In [83]:
germ_data.drop(germ_data[germ_data['Fuel Type'] == '\xa0DIESEL'].index,inplace=True)
germ_data.drop(germ_data[germ_data['Fuel Type'] == 'HYBRID LPG/PHEV'].index,inplace=True)
germ_data.drop(germ_data[germ_data['Fuel Type'] == 'MULTI-SUBSTANCE'].index,inplace=True)
germ_data.drop(germ_data[germ_data['Fuel Type'] == 'NATURAL GAS'].index,inplace=True)
germ_data.drop(germ_data[germ_data['Fuel Type'] == 'LPG'].index,inplace=True)
germ_data.drop(germ_data[germ_data['Fuel Type'] == 'HYBRID LPG/ELECTRIC'].index,inplace=True)

In [84]:
germ_data['Fuel Type'].unique()

array(['PETROL', 'HYBRID PETROL/ELECTRIC', 'ELECTRIC',
       'PETROL/COMPRESSED NATURAL GAS', 'PETROL/LIQUID GAS',
       'HYBRID PETROL/PHEV', 'HYBRID HYDROGEN/PHEV',
       'HYBRID DIESEL/ELECTRIC', 'HYDROGEN/FC', 'HYBRID HYDROGEN/FC/PHEV'],
      dtype=object)

In [85]:
germ_data.replace({'HYBRID PETROL/ELECTRIC': 'PETROL',
                  'PETROL/COMPRESSED NATURAL GAS':'PETROL',
                  'PETROL/LIQUID GAS':'PETROL',
                  'HYBRID PETROL/PHEV':'PETROL',
                  'HYBRID HYDROGEN/PHEV':'HYDROGEN',
                  'HYBRID DIESEL/ELECTRIC':'DIESEL',
                  'HYDROGEN/FC':'HYDROGEN',
                  'HYBRID HYDROGEN/FC/PHEV':'HYDROGEN',
                  },inplace=True)

In [86]:
for i in uk_data['Fuel Type'].unique():
    for j in germ_data['Fuel Type']:
        if i in j:
            germ_data['Fuel Type'] = germ_data['Fuel Type'].replace (j,i)

In [18]:
germ_data = germ_data.dropna(subset=['Manufacturer'])

In [19]:
germ_data['Manufacturer'] = germ_data['Manufacturer'].str.upper()

In [20]:
for i in uk_data['Manufacturer'].unique():
    for j in germ_data['Manufacturer']:
        if i in j:
            germ_data['Manufacturer'] = germ_data['Manufacturer'].replace (j,i)

In [21]:
uk_data.shape

(17856, 28)

In [22]:
germ_data.shape

(10652, 16)

In [23]:
germ_data = germ_data.loc[(germ_data['max. Length'] >=1450) & (germ_data['max. Length'] <=6000)]

In [24]:
germ_data = germ_data.loc[(germ_data['max. Width'] >=1372) & (germ_data['max. Width'] <=2650)]

In [25]:
germ_data = germ_data.loc[(germ_data['max. Gross Vehicle Weight'] >=1148) & (germ_data['max. Gross Vehicle Weight'] <=4000)]

In [26]:
germ_data.shape

(8043, 16)

In [31]:
# germ_data.to_csv('23.csv')

In [27]:
# importing the ger-co2 file
germ_data_modelname=pd.read_excel(r'ger-co2.xlsx', sheet_name= 'mapped_ger')

In [55]:
germ_data_modelname.head(5)

,Unnamed: 0,Manufacturer Identification Number,Model Identification Numer,No. Of Driving Axles,max. Length,max. Width,max. Height,max. Gross Vehicle Weight,Manufacturer,Commercial Name,Type,Wheelbase,Power/Weight Ratio,Seats,Fuel Code,max. Top Speed,Fuel Type,Commercial Name New
0,40,5,AAA,1.0,4091,1781,1299,1640,BMW,Z Reihe,Z85,2495.0,0.139286,2,1,250.0,PETROL,Z Reihe
1,41,5,AAB,1.0,4091,1781,1299,1600,BMW,Z Reihe,Z85,2495.0,0.095588,2,1,229.0,PETROL,Z Reihe
2,42,5,AAC,1.0,4091,1781,1299,1620,BMW,Z Reihe,Z85,2495.0,0.116152,2,1,240.0,PETROL,Z Reihe
3,43,5,AAD,1.0,4091,1781,1299,1620,BMW,Z Reihe,Z85,2495.0,0.112523,2,1,240.0,PETROL,Z Reihe
4,44,5,AAE,1.0,4239,1748,1421,1800,BMW,1er Reihe,187,2660.0,0.077348,5,1,210.0,PETROL,NaN


In [56]:
co2_data=pd.read_excel(r'ger-co2.xlsx', sheet_name= 'just_co2')
co2_data1 = co2_data.dropna(how='all')

In [57]:
co2_data1.tail(2)
co2_data1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 53655 entries, 0 to 53654
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   sl               53655 non-null  int64  
 1   Type             51081 non-null  object 
 2   Make             36495 non-null  object 
 3   commercial name  50336 non-null  object 
 4   Fuel Type        53655 non-null  object 
 5   Enedc (g/km)     51574 non-null  float64
 6   Ewltp (g/km)     0 non-null      float64
dtypes: float64(2), int64(1), object(4)
memory usage: 3.3+ MB


In [58]:
# aggregate co2 data by Type, Make, commercial name, get average of CO2
co2_data_agg = co2_data.groupby(['Type','Make','commercial name'])['Enedc (g/km)'].mean().reset_index()
# co2_data.groupby(['Type','Make','commercial name']).agg(avg_co2= ('Enedc (g/km)'))
co2_data_agg.head(2)

,Type,Make,commercial name,Enedc (g/km)
0,0,CITROEN,C4 CACTUS,99.066667
1,0,PEUGEOT,5008,139.295455


In [60]:
co2_data_agg[co2_data_agg['Type']=='Z85']

,Type,Make,commercial name,Enedc (g/km)
8469,Z85,BMW,Z Reihe,198.333333
8470,Z85,BMW,ZREIHE,NaN


In [61]:
germ_data_modelname[germ_data_modelname['Type']=='Z85']

,Unnamed: 0,Manufacturer Identification Number,Model Identification Numer,No. Of Driving Axles,max. Length,max. Width,max. Height,max. Gross Vehicle Weight,Manufacturer,Commercial Name,Type,Wheelbase,Power/Weight Ratio,Seats,Fuel Code,max. Top Speed,Fuel Type,Commercial Name New
0,40,5,AAA,1.0,4091,1781,1299,1640,BMW,Z Reihe,Z85,2495.0,0.139286,2,1,250.0,PETROL,Z Reihe
1,41,5,AAB,1.0,4091,1781,1299,1600,BMW,Z Reihe,Z85,2495.0,0.095588,2,1,229.0,PETROL,Z Reihe
2,42,5,AAC,1.0,4091,1781,1299,1620,BMW,Z Reihe,Z85,2495.0,0.116152,2,1,240.0,PETROL,Z Reihe
3,43,5,AAD,1.0,4091,1781,1299,1620,BMW,Z Reihe,Z85,2495.0,0.112523,2,1,240.0,PETROL,Z Reihe
5,45,5,AAF,1.0,4091,1781,1283,1680,BMW,Z Reihe,Z85,2495.0,0.138298,2,1,250.0,PETROL,Z Reihe
10,59,5,AAT,1.0,4091,1781,1299,1520,BMW,Z Reihe,Z85,2495.0,0.084942,2,1,220.0,PETROL,Z Reihe


In [62]:
co2_data_agg.size

34128

In [63]:
co2_data_agg.rename(columns={'Make': 'Manufacturer', 'commercial name':'Commercial Name New'}, inplace=True)
co2_data_agg.head(20)

,Type,Manufacturer,Commercial Name New,Enedc (g/km)
0,0,CITROEN,C4 CACTUS,99.066667
1,0,PEUGEOT,5008,139.295455
2,2,CITROEN,C3 AIRCROSS,129.000000
3,2,PEUGEOT,206+,125.333333
4,2,RENAULT,CAPTUR,144.000000
5,3,,C4PICASSO,117.000000
6,3,CITROEN,C4 GRAND PICASSO,117.000000
7,3,CITROEN,C4 PICASSO,117.838710
8,4,,A6LIMOUSINE2.0TFSI,166.000000
9,4,,A6LIMOUSINE3.0TDI,156.000000


In [87]:
germ_data_2 = pd.merge(germ_data_modelname, co2_data_agg, how='left', on=['Type', 'Manufacturer', 'Commercial Name New'])

In [88]:
germ_data_2.shape

(8043, 19)

In [89]:
germ_data_2.to_csv('germ_with_co2.csv')